# **Compilación con C**

In [2]:
from datetime import date
import pandas as pd
import random
import numpy as np 
import networkx as nx
import time

def get_data(data_dir, date=date.today().strftime("%Y-%m-%d"), tam_data=100, price='Open'):
    """
    Obtiene datos de un directorio de una fecha dada
    param:
    data_dir: directorio de datos
    date: fecha a analizar, por default toma la fecha actual si no se especifica
    tam_data: analiza cierto tamaño del dataset de forma aleatoria, por default son 100 monedas a analizar
    price: selecciona con que precio se hará el ejercicio(High,Low,Open,Close), por default es Open
    return:
    dataframe: con datos especificados
    """
    df = pd.read_csv(data_dir)
    df_date = df[df['Date']==date]
    df_date = df_date[df_date[price] > 0]
    
    if(tam_data == None):
        df_random = df_date
    else:
        df_random = df_date.sample(n = tam_data)
    
    
    df_random = df_random.reset_index()
    data = df_random[["ticker", price]]
    data.columns = ['Símbolo', 'Precio']
    
    return data    

def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
    c1.drop(columns=['Precio'],inplace=True)
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1
    return c1

def log_transformed_rep(data):
    """
    Log-Transformed Representations
    param:
        dataframe
    return:
        dataframe
    """
    
    df_ln = np.round(-np.log(data),2)
    return df_ln

def create_grap(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

**Función a compilar**

In [3]:
def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:  
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

In [3]:
#df = get_data('../data/historical_data.csv', '2022-05-12')
df = get_data('data/historical_data.csv', '2022-05-12',5)
df.head(15)

,Símbolo,Precio
0,MIC,0.049520
1,IBZ,0.000110
2,BURN,0.000019
3,ONION,0.081471
4,OUSE,0.000002


In [5]:
G = create_grap(df)

In [6]:
%timeit bf_negative_cycle(G,0)

2.7 s ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   36 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           58
Model name:                      Intel(R) Core(TM) i5-3210M CPU @ 2.50GHz
Stepping:                        9
CPU MHz:                         2694.743
CPU max MHz:                     3100.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4988.79
Virtualization:                  VT-x
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        512 KiB
L3 cache:                       

In [19]:
#%%bash
#sudo lshw -C memory

In [20]:
%%bash
uname -ar #r for kernel, a for all

Linux 5a4ea55e41da 5.13.0-41-generic #46~20.04.1-Ubuntu SMP Wed Apr 20 13:16:21 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


# Cython

In [7]:
%%file bf_cython.pyx
import numpy as np 
def bf_negative_cycle_p(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
        
    return cycle.reverse()

Overwriting bf_cython.pyx


In [8]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("bf_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Overwriting setup.py


In [9]:
%%bash
python3 setup.py build_ext --inplace

Compiling bf_cython.pyx because it changed.
[1/1] Cythonizing bf_cython.pyx
running build_ext
building 'bf_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c bf_cython.c -o build/temp.linux-x86_64-3.8/bf_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/bf_cython.o -o /datos/maestria_2022p/Opt2/notebooks/bf_cython.cpython-38-x86_64-linux-gnu.so


**Importando**

In [10]:
import bf_cython
start_time = time.time()
res = bf_cython.bf_negative_cycle_p(G)
end_time = time.time()

In [11]:
secs = end_time-start_time
print("Bellman Ford tomó",secs,"segundos" )

Bellman Ford tomó 1.9562475681304932 segundos


In [12]:
%load_ext Cython

In [13]:
%%cython
import numpy as np 
def bf_negative_cycle_cc(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values

    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]    
    
    return cycle.reverse()

In [15]:
start_time = time.time()
res = bf_negative_cycle_cc(G)
end_time = time.time()
secs = end_time-start_time
print("BF tomó",secs,"segundos" )

BF tomó 2.082916021347046 segundos


In [16]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_cython.pyx

In [17]:
from IPython.display import display, HTML, Image

In [18]:
display(HTML("bf_cython.html"))

In [23]:
%%file bf_cython2.pyx
import numpy as np 
def bf_negative_cycle_cc(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

Overwriting bf_cython2.pyx


In [24]:
start_time = time.time()
res = bf_negative_cycle_cc(G)
end_time = time.time()
secs = end_time-start_time
print("BF tomó",secs,"segundos" )

BF tomó 1.96793794631958 segundos


In [25]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_cython2.pyx

In [26]:
display(HTML("bf_cython2.html"))

In [63]:
print(res)

[99, 98, 99]


# Cython y OpenMP

In [27]:
%%file bf_cython_openmp.pyx
from cython.parallel import prange
from libc.math cimport exp as c_exp
import numpy as np 

def bf_negative_cycle_cc(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

Overwriting bf_cython_openmp.pyx


In [28]:
%%bash
$HOME/.local/bin/cython -3 --force bf_cython_openmp.pyx

In [29]:
%%file setup_openmp.py
from setuptools import Extension, setup
from Cython.Build import cythonize

ext_modules = [Extension("bf_cython_openmp",
                         ["bf_cython_openmp.pyx"], 
                         extra_compile_args=["-fopenmp"],
                         extra_link_args=["-fopenmp"],
                        )
              ]

setup(ext_modules = cythonize(ext_modules))

Overwriting setup_openmp.py


In [30]:
%%bash
python3 setup_openmp.py build_ext --inplace

running build_ext
building 'bf_cython_openmp' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c bf_cython_openmp.c -o build/temp.linux-x86_64-3.8/bf_cython_openmp.o -fopenmp
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/bf_cython_openmp.o -o build/lib.linux-x86_64-3.8/bf_cython_openmp.cpython-38-x86_64-linux-gnu.so -fopenmp
copying build/lib.linux-x86_64-3.8/bf_cython_openmp.cpython-38-x86_64-linux-gnu.so -> 


In [31]:
import bf_cython_openmp

In [32]:
start_time = time.time()
res_openmp = bf_cython_openmp.bf_negative_cycle_cc(G)
end_time = time.time()
secs = end_time-start_time
print("BF_openmp tomó",secs,"segundos" )

BF_openmp tomó 1.1095116138458252 segundos


#### Perfilando código de exchange matrix

In [66]:
n = df.shape[0]
# maximum bid-ask spread in pct of bid, 0.05 for 5%
max_spread_pct = 0.05 
c1 = df[['Precio']]
c1
c2 = df[['Precio']]
c3 = df[['Precio']]
aux = c1.copy()
random.seed(10)
c1

,Precio
0,0.049520
1,0.000110
2,0.000019
3,0.081471
4,0.000002


In [62]:
for i in range(n):
    print(c1[['Precio']].values[i])
    print(i)
    x = c1.loc[i]['Precio']
    print(x)
    a = 1+random.uniform(0,max_spread_pct)
    #print(a)
    #print( df.iloc[:, 1])
    c1[i] = aux/c1[['Precio']].values[i]*(a)
    c2[i] = aux/x*(a)
    print('c1[i]:',c1[i])
    print('c2[i]:',c2[i])
    print('c1:',c1)
    print('c2:',c2)

[0.04952]
0
0.04952000081539154
c1[i]: 0    1.028570
1    0.002285
2    0.000395
3    1.692218
4    0.000042
Name: 0, dtype: float64
c2[i]: 0    1.028570
1    0.002285
2    0.000395
3    1.692218
4    0.000042
Name: 0, dtype: float64
c1:      Precio         0
0  0.049520  1.028570
1  0.000110  0.002285
2  0.000019  0.000395
3  0.081471  1.692218
4  0.000002  0.000042
c2:      Precio         0
0  0.049520  1.028570
1  0.000110  0.002285
2  0.000019  0.000395
3  0.081471  1.692218
4  0.000002  0.000042
[0.00011]
1
0.00011000000085914508
c1[i]: 0    459.835725
1      1.021444
2      0.176431
3    756.528218
4      0.018572
Name: 1, dtype: float64
c2[i]: 0    459.835725
1      1.021444
2      0.176431
3    756.528218
4      0.018572
Name: 1, dtype: float64
c1:      Precio         0           1
0  0.049520  1.028570  459.835725
1  0.000110  0.002285    1.021444
2  0.000019  0.000395    0.176431
3  0.081471  1.692218  756.528218
4  0.000002  0.000042    0.018572
c2:      Precio         0    

In [67]:
for i in range(n):
    c3[i] = aux/c3.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
    
c3

,Precio,0,1,2,3,4
0,0.049520,1.028570,459.835725,2681.650377,0.614087,25766.892199
1,0.000110,0.002285,1.021444,5.956816,0.001364,57.236634
2,0.000019,0.000395,0.176431,1.028905,0.000236,9.886327
3,0.081471,1.692218,756.528218,4411.889013,1.010305,42392.054368
4,0.000002,0.000042,0.018572,0.108306,0.000025,1.040666


In [39]:
n = df.shape[0]
# maximum bid-ask spread in pct of bid, 0.05 for 5%
max_spread_pct = 0.05 

c1 = df[['Precio']]
aux = c1.copy()
random.seed(10)
for i in range(n):
    c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
c1.drop(columns=['Precio'],inplace=True)
for i in range(len(c1.index)):
    for j in range(len(c1.columns)):
        if i==j:
            c1.loc[i,j] = 1
c1

,0,1,2,3,4
0,1.000000e+00,14936.270650,6.769266e+06,158556.959703,7.849115e+06
1,7.034067e-05,1.000000,4.629288e+02,10.843211,5.367762e+02
2,1.563390e-07,0.002270,1.000000e+00,0.024100,1.193038e+00
3,6.553919e-06,0.095172,4.313291e+01,1.000000,5.001357e+01
4,1.363718e-07,0.001980,8.974955e-01,0.021022,1.000000e+00


In [63]:
start_time = time.time()
d = exchange_rate_matrix(df)
end_time = time.time()
secs = end_time-start_time
print("exchange_rate_matrix tomó: ",secs,"segundos" )

exchange_rate_matrix tomó:  0.09999990463256836 segundos


In [68]:
def exchange_rate_matrix2(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
        
    c1.drop(columns=['Precio'],inplace=True)
    
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1    
    return c1

In [69]:
start_time = time.time()
d2 = exchange_rate_matrix2(df)
end_time = time.time()
secs = end_time-start_time
print("exchange_rate_matrix tomó: ",secs,"segundos" )

exchange_rate_matrix tomó:  0.01100015640258789 segundos


In [70]:
%%file ex_mat.pyx
import random
def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
    c1.drop(columns=['Precio'],inplace=True)
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1
    return c1

Writing ex_mat.pyx


In [ ]:
start_time = time.time()
res = exchange_rate_matrix(df)
end_time = time.time()
secs = end_time-start_time
print("exchange_rate_matrix tomó: ",secs,"segundos" )

In [71]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate ex_mat.pyx

bash: line 1: /c/Users/aide.gonzalezc/.local/bin/cython: No such file or directory


CalledProcessError: Command 'b'$HOME/.local/bin/cython --force -3 --annotate ex_mat.pyx\n'' returned non-zero exit status 127.

In [ ]:
display(HTML("ex_mat.html"))

In [ ]:
%%file ex_mat2.pyx
import random
def exchange_rate_matrix2(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
        
    c1.drop(columns=['Precio'],inplace=True)
    
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1    
    return c1

In [ ]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate ex_mat2.pyx

In [ ]:
display(HTML("ex_mat2.html"))

In [ ]:
%%file ex_mat_openmp.pyx
from cython.parallel import prange
from libc.math cimport exp as c_exp
import numpy as np 

def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
        
    c1.drop(columns=['Precio'],inplace=True)
    
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1    
    return c1

In [ ]:
%%bash
$HOME/.local/bin/cython -3 --force ex_mat_openmp.pyx

In [ ]:
%%file setup_openmp.py
from setuptools import Extension, setup
from Cython.Build import cythonize

ext_modules = [Extension("ex_mat_openmp",
                         ["ex_mat_openmp.pyx"], 
                         extra_compile_args=["-fopenmp"],
                         extra_link_args=["-fopenmp"],
                        )
              ]

setup(ext_modules = cythonize(ext_modules))

In [ ]:
%%bash
python3 setup_openmp.py build_ext --inplace

In [ ]:
import ex_mat_openmp

In [ ]:
start_time = time.time()
d = ex_mat_openmp.exchange_rate_matrix(df)
end_time = time.time()
secs = end_time-start_time
print("exchange_rate_matrix tomó: ",secs,"segundos" )